In [11]:
import pandas as pd

df = pd.read_csv(r"EFX_dataset25.csv")
df = df[["EFX_LEGSUBNAMEALL", "EFX_LEGDOMULTNAMEALL", "EFX_AFFLULTNAMEALL"]]

In [12]:
# 'Root' = 'EFX_LEGDOMULTNAMEALL'
# 'Child' = 'EFX_LEGDOMULTNAMEALL' 'EFX_LEGSUBNAMEALL'
# 'EFX_LEGSUBNAMEALL' --> 'EFX_LEGDOMULTNAMEALL' --> 'EFX_AFFLULTNAMEALL'

In [13]:
import plotly.graph_objects as go
import networkx as nx
import pandas as pd

# # Sample DataFrame
# data = {
#     "EFX_LEGSUBNAMEALL": [
#         "CIS",
#         "DOBLIN INC",
#         "HEAT INDUSTRIES",
#         "CIS",
#         "HEAT INDUSTRIES",
#     ],
#     "EFX_LEGDOMULTNAMEALL": ["DELOITTE US DENVER"] * 5,
#     "EFX_AFFLULTNAMEALL": ["DELOITTE TOUCHE TOHMATSU LIMITED"] * 5,
# }
# df = pd.DataFrame(data)

# Create a directed graph
G = nx.DiGraph()

# Add edges to the graph
for _, row in df.iterrows():
    G.add_edge(row["EFX_AFFLULTNAMEALL"], row["EFX_LEGDOMULTNAMEALL"])
    G.add_edge(row["EFX_LEGDOMULTNAMEALL"], row["EFX_LEGSUBNAMEALL"])

# Generate hierarchical layout using Graphviz
pos = nx.nx_agraph.graphviz_layout(G, prog="dot")

# Extract edge coordinates
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# Extract node coordinates
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]

# Create figure
fig = go.Figure()

# Add edges
fig.add_trace(
    go.Scatter(
        x=edge_x,
        y=edge_y,
        mode="lines",
        line=dict(width=1, color="black"),
        hoverinfo="none",
    )
)

# Add nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        marker=dict(size=10, color="lightblue"),
        text=list(G.nodes()),
        textposition="top center",
    )
)

# Layout settings
fig.update_layout(
    title="Hierarchy Graph",
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
)

fig.show()